# Gateway Swap Router Example

This notebook demonstrates how to use the Gateway Swap router for DEX swap operations.

The Gateway Swap router provides functionality for:
- Getting swap quotes (price discovery)
- Executing swaps on DEX routers (Jupiter, 0x)
- Tracking swap history and status
- Viewing swap statistics

## Setup

Initialize the Hummingbot API client.

In [ ]:
from hummingbot_api_client import HummingbotAPIClient
from decimal import Decimal

client = HummingbotAPIClient(base_url='http://localhost:8000')
await client.init()

## Get Swap Quote

Get a price quote for a swap without executing the transaction.

In [ ]:
# Get quote for buying 1 SOL with USDC on Jupiter
quote = await client.gateway_swap.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    side='BUY',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')  # 1% slippage
)
quote

In [ ]:
# Display quote information
print(f"Trading Pair: {quote['base']}/{quote['quote']}")
print(f"Price: ${quote['price']}")
print(f"Amount In: {quote['amount_in']} {quote['quote']}")
print(f"Amount Out: {quote['amount_out']} {quote['base']}")
print(f"Slippage: {quote['slippage_pct']}%")
if quote.get('gas_estimate'):
    print(f"Gas Estimate: {quote['gas_estimate']}")

In [ ]:
# Get quote for selling 1 SOL for USDC
quote_sell = await client.gateway_swap.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    side='SELL',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')
)

print(f"Selling 1 SOL:")
print(f"You will receive: ~{quote_sell['amount_out']} USDC")
print(f"Price: ${quote_sell['price']} per SOL")

## Swap with Custom Tokens

Get quotes for swaps involving custom tokens.

In [ ]:
# Get quote for buying custom token with USDC
# Note: Make sure you added the token first using the gateway router
quote_custom = await client.gateway_swap.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='BONK-USDC',
    side='BUY',
    amount=Decimal('1000000'),  # Buy 1M BONK
    slippage_pct=Decimal('2.0')  # 2% slippage for smaller tokens
)

print(f"Buying 1M BONK:")
print(f"You will pay: ~{quote_custom['amount_in']} USDC")
print(f"Price: ${quote_custom['price']} per BONK")

## Execute Swap

Execute an actual swap transaction on the DEX.

**⚠️ WARNING:** This will execute a real transaction on-chain and costs gas fees!

In [ ]:
# Execute a small swap (uncomment to run)
# result = await client.gateway_swap.execute_swap(
#     connector='jupiter',
#     network='solana-mainnet-beta',
#     trading_pair='USDT-USDC',
#     side='BUY',
#     amount=Decimal('1'),  # Buy 1 USDT
#     slippage_pct=Decimal('1.0'),
# )
#
# print(f"Transaction Hash: {result['transaction_hash']}")
# print(f"Status: {result['status']}")
# print(f"Trading Pair: {result['trading_pair']}")
# print(f"Side: {result['side']}")
# print(f"Amount: {result['amount']}")

## Check Swap Status

Check the status of a previously executed swap.

In [ ]:
# Check swap status by transaction hash
# tx_hash = 'YOUR_TRANSACTION_HASH_HERE'
# swap_status = await client.gateway_swap.get_swap_status(tx_hash)
#
# print(f"Transaction: {swap_status['transaction_hash']}")
# print(f"Status: {swap_status['status']}")
# print(f"Trading Pair: {swap_status['trading_pair']}")
# print(f"Side: {swap_status['side']}")
# print(f"Input Amount: {swap_status['input_amount']}")
# print(f"Output Amount: {swap_status['output_amount']}")
# print(f"Price: {swap_status['price']}")

## Search Swap History

Search through your swap history with various filters.

In [ ]:
# Search all swaps on Solana
swaps = await client.gateway_swap.search_swaps(
    network='solana-mainnet-beta',
    connector='jupiter',
    limit=10
)

print(f"Found {len(swaps['data'])} swaps")
for swap in swaps['data']:
    print(f"\n{swap['trading_pair']} - {swap['side']}")
    print(f"  Amount: {swap['input_amount']} → {swap['output_amount']}")
    print(f"  Status: {swap['status']}")
    print(f"  TX: {swap['transaction_hash'][:16]}...")

In [ ]:
# Search swaps for a specific trading pair
sol_swaps = await client.gateway_swap.search_swaps(
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    status='CONFIRMED',
    limit=5
)

print(f"Recent SOL-USDC swaps:")
for swap in sol_swaps['data']:
    print(f"  {swap['side']}: {swap['input_amount']} → {swap['output_amount']}")

In [ ]:
# Search swaps by wallet address
# wallet_swaps = await client.gateway_swap.search_swaps(
#     wallet_address='YOUR_WALLET_ADDRESS',
#     limit=20
# )
# 
# print(f"Swaps for wallet: {len(wallet_swaps['data'])} total")

## Get Swap Summary Statistics

Get aggregated statistics about your swaps.

In [ ]:
# Get summary for all swaps on Solana
summary = await client.gateway_swap.get_swaps_summary(
    network='solana-mainnet-beta'
)

print(f"Swap Summary for Solana:")
print(f"  Total Swaps: {summary.get('total_swaps', 0)}")
print(f"  Total Volume: ${summary.get('total_volume', 0):.2f}")
print(f"  Success Rate: {summary.get('success_rate', 0):.1f}%")
print(f"  Average Trade Size: ${summary.get('avg_trade_size', 0):.2f}")
if 'total_fees' in summary:
    print(f"  Total Fees: ${summary['total_fees']:.2f}")

In [ ]:
# Get summary for a specific wallet
# wallet_summary = await client.gateway_swap.get_swaps_summary(
#     wallet_address='YOUR_WALLET_ADDRESS',
#     network='solana-mainnet-beta'
# )
# 
# print(f"Your Trading Stats:")
# print(f"  Total Trades: {wallet_summary['total_swaps']}")
# print(f"  Volume: ${wallet_summary['total_volume']:.2f}")
# print(f"  Success Rate: {wallet_summary['success_rate']:.1f}%")

## Example Workflow: Quote and Execute

Complete workflow showing how to get a quote and then execute if the price is acceptable.

In [ ]:
# 1. Get quote
print("Step 1: Getting quote...")
quote = await client.gateway_swap.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    side='BUY',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')
)

print(f"Quote received:")
print(f"  Price: ${quote['price']}")
print(f"  You will pay: {quote['amount_in']} USDC")
print(f"  You will receive: {quote['amount_out']} SOL")

# 2. Check if price is acceptable
max_price = Decimal('200')  # Maximum price you're willing to pay
quote_price = Decimal(str(quote['price']))

print(f"\nStep 2: Checking price...")
if quote_price <= max_price:
    print(f"✅ Price ${quote_price} is acceptable (max: ${max_price})")
    print("\nStep 3: Ready to execute swap (uncomment below to execute)")

    # 3. Execute swap (uncomment to run)
    # result = await client.gateway_swap.execute_swap(
    #     connector='jupiter',
    #     network='solana-mainnet-beta',
    #     trading_pair='SOL-USDC',
    #     side='BUY',
    #     amount=Decimal('1'),
    #     slippage_pct=Decimal('1.0')
    # )
    # print(f"\n✅ Swap executed!")
    # print(f"Transaction: {result['transaction_hash']}")
else:
    print(f"❌ Price ${quote_price} is too high (max: ${max_price})")
    print("Swap cancelled")

## Using Other DEX Connectors

The same methods work with other DEX routers like 0x on Ethereum networks.

In [ ]:
# Example: Get quote on Ethereum using 0x
# quote_eth = await client.gateway_swap.get_swap_quote(
#     connector='0x',
#     network='ethereum-mainnet',
#     trading_pair='ETH-USDC',
#     side='BUY',
#     amount=Decimal('1'),
#     slippage_pct=Decimal('1.0')
# )
# quote_eth

## Tips and Best Practices

1. **Always get a quote first** - Check the price and expected amounts before executing
2. **Set appropriate slippage** - Higher slippage for volatile/illiquid tokens (2-5%), lower for stable pairs (0.5-1%)
3. **Check gas estimates** - Make sure you have enough native tokens for gas fees
4. **Use wallet_address parameter** - Specify which wallet to use if you have multiple configured
5. **Monitor transaction status** - Save the transaction hash and check status with `get_swap_status()`
6. **Start with small amounts** - Test with small trades before executing large swaps
7. **Review swap history** - Use `search_swaps()` to track your trading activity
8. **Check statistics** - Use `get_swaps_summary()` to analyze your trading performance

## Cleanup

Close the client when done.

In [ ]:
await client.close()